In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torchvision import models

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import torch.optim as optim
from tqdm import tqdm

c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
IMG_SIZE = 224
BATCH_SIZE = 32

In [3]:

transform_train = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [4]:
train_data = datasets.ImageFolder(root='data/CatDogDataset/train', transform=transform_train)
val_data = datasets.ImageFolder(root='data/CatDogDataset/val', transform=transform_val)
test_data = datasets.ImageFolder(root='data/CatDogDataset/test', transform=transform_val)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)    
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
class_names = train_data.classes
print(class_names)  # ['Cat', 'Dog']

['Cat', 'Dog']


In [6]:
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, 1)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\LENOVO/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 34.4MB/s]


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_model(model, train_loader, val_loader, epochs=5, lr=1e-4):
    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        running_loss, running_correct, running_total = 0, 0, 0
        loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}]")

        for imgs, labels in loop:
            imgs, labels = imgs.to(device), labels.unsqueeze(1).float().to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Tính accuracy trên batch hiện tại
            preds = torch.sigmoid(outputs) > 0.5
            running_correct += (preds == labels).sum().item()
            running_total += labels.size(0)
            running_loss += loss.item()

            batch_acc = 100 * running_correct / running_total
            loop.set_postfix(loss=loss.item(), acc=batch_acc)

        # Tính train accuracy toàn epoch
        train_acc = 100 * running_correct / running_total
        train_loss = running_loss / len(train_loader)

        # Tính val accuracy
        val_acc = evaluate(model, val_loader)

        print(f"\nEpoch {epoch+1}/{epochs} ✅ | "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%\n")

    return model


In [9]:
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.unsqueeze(1).float().to(device)
        outputs = model(imgs)
        preds = torch.sigmoid(outputs) > 0.5
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return 100 * correct / total


In [10]:
model = train_model(model, train_loader, val_loader, epochs=5)

Epoch [1/5]:  89%|████████▊ | 485/548 [30:49<04:03,  3.87s/it, acc=97.1, loss=0.00422]c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Epoch [1/5]: 100%|██████████| 548/548 [34:31<00:00,  3.78s/it, acc=97.1, loss=0.0503] 



Epoch 1/5 ✅ | Train Loss: 0.0788 | Train Acc: 97.13% | Val Acc: 98.54%



Epoch [2/5]: 100%|██████████| 548/548 [1:33:23<00:00, 10.23s/it, acc=98.6, loss=0.00966]     



Epoch 2/5 ✅ | Train Loss: 0.0379 | Train Acc: 98.58% | Val Acc: 98.86%



Epoch [3/5]: 100%|██████████| 548/548 [34:56<00:00,  3.83s/it, acc=99, loss=0.00361]   



Epoch 3/5 ✅ | Train Loss: 0.0287 | Train Acc: 99.00% | Val Acc: 98.68%



Epoch [4/5]: 100%|██████████| 548/548 [33:46<00:00,  3.70s/it, acc=99.3, loss=0.00313] 



Epoch 4/5 ✅ | Train Loss: 0.0195 | Train Acc: 99.27% | Val Acc: 98.82%



Epoch [5/5]: 100%|██████████| 548/548 [33:06<00:00,  3.62s/it, acc=99.3, loss=0.151]   



Epoch 5/5 ✅ | Train Loss: 0.0187 | Train Acc: 99.30% | Val Acc: 99.14%



In [11]:
torch.save(model.state_dict(), "mobilenet_weight.pth")

In [12]:
test_acc = evaluate(model, test_loader)
test_acc

98.88089528377299